In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')

import PETsARD

In [2]:
from importlib import resources

import yaml

# organized yaml
PETSARD_CONFIG = {}
LIST_YAML = [
    ('loader_config.yaml', 'loader'),
    ('benchmark_datasets.yaml', 'benchmark_datasets')
]
for yaml_name, config_name in LIST_YAML:
    with resources.open_text('PETsARD.Loader', yaml_name) as file:
        PETSARD_CONFIG[config_name] = yaml.safe_load(file)

from pprint import pprint
pprint(PETSARD_CONFIG)

{'benchmark_datasets': {'adult': {'filename': 'adult.csv',
                                  'public': 'public',
                                  'sha256': '1f13ee2bf9d7c66098429281ab91fa1b51cbabd3b805cc365b3c6b44491ea2c0'},
                        'adult-private': {'filename': 'adult_private_for_demo.csv',
                                          'public': 'private',
                                          'sha256': '1f13ee2bf9d7c66098429281ab91fa1b51cbabd3b805cc365b3c6b44491ea2c0'}},
 'loader': {'benchmark_bucket_name': {'private': 'petsard-benchmark-private',
                                      'public': 'petsard-benchmark'},
            'benchmark_region_name': 'ap-southeast-1',
            'file_ext': {'LoaderPandasCsv': ['csv'],
                         'LoaderPandasExcel': ['xls',
                                               'xlsx',
                                               'xlsm',
                                               'xlsb',
                              

In [3]:
REGION_NAME = PETSARD_CONFIG['loader']['benchmark_region_name']
BUCKET_NAME = PETSARD_CONFIG['loader']['benchmark_bucket_name']
PETSARD_CONFIG['benchmark_datasets'] = {
    key: {
        'filename':    value['filename'],
        'public':      value['public'],
        'region_name': REGION_NAME,
        'bucket_name': BUCKET_NAME[value['public']],
        'sha256':      value['sha256']
    }
    for key, value in PETSARD_CONFIG['benchmark_datasets'].items()
}

pprint(PETSARD_CONFIG['benchmark_datasets'])

{'adult': {'bucket_name': 'petsard-benchmark',
           'filename': 'adult.csv',
           'public': 'public',
           'region_name': 'ap-southeast-1',
           'sha256': '1f13ee2bf9d7c66098429281ab91fa1b51cbabd3b805cc365b3c6b44491ea2c0'},
 'adult-private': {'bucket_name': 'petsard-benchmark-private',
                   'filename': 'adult_private_for_demo.csv',
                   'public': 'private',
                   'region_name': 'ap-southeast-1',
                   'sha256': '1f13ee2bf9d7c66098429281ab91fa1b51cbabd3b805cc365b3c6b44491ea2c0'}}


In [4]:
PETSARD_CONFIG['loader']['file_ext']

{'LoaderPandasCsv': ['csv'],
 'LoaderPandasExcel': ['xls', 'xlsx', 'xlsm', 'xlsb', 'odf', 'ods', 'odt']}

In [5]:
from PETsARD.Loader.LoaderPandas import (
    LoaderPandasCsv,
    LoaderPandasExcel,
)

FILE_EXT = PETSARD_CONFIG['loader']['file_ext']
LOADER_MAP = {
    'LoaderPandasCsv': LoaderPandasCsv,
    'LoaderPandasExcel': LoaderPandasExcel
}
PETSARD_CONFIG['loader']['Loader'] = {
    file_ext: LOADER_MAP[key]
    for key, file_exts in FILE_EXT.items()
    for file_ext in file_exts
}

pprint(PETSARD_CONFIG)

{'benchmark_datasets': {'adult': {'bucket_name': 'petsard-benchmark',
                                  'filename': 'adult.csv',
                                  'public': 'public',
                                  'region_name': 'ap-southeast-1',
                                  'sha256': '1f13ee2bf9d7c66098429281ab91fa1b51cbabd3b805cc365b3c6b44491ea2c0'},
                        'adult-private': {'bucket_name': 'petsard-benchmark-private',
                                          'filename': 'adult_private_for_demo.csv',
                                          'public': 'private',
                                          'region_name': 'ap-southeast-1',
                                          'sha256': '1f13ee2bf9d7c66098429281ab91fa1b51cbabd3b805cc365b3c6b44491ea2c0'}},
 'loader': {'Loader': {'csv': <class 'PETsARD.Loader.LoaderPandas.LoaderPandasCsv'>,
                       'odf': <class 'PETsARD.Loader.LoaderPandas.LoaderPandasExcel'>,
                       'ods': <clas

In [6]:
loader = PETsARD.Loader(
    filepath='../[sunset]/data/[Adt Income] adult.csv',
    na_values={
        k: '?' for k in [
            'workclass',
            'occupation',
            'native-country'
        ]
    }
)
print(loader.data.head(1))


   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [7]:
# import boto3
# s3 = boto3.client('s3')
# for bucket in s3.list_buckets()["Buckets"]:
#     print(bucket["Name"])

In [8]:
# import hashlib


# def calculate_sha256(file_path):
#     sha256_hash = hashlib.sha256()
#     with open(file_path, "rb") as f:
#         for byte_block in iter(lambda: f.read(4096), b""):
#             sha256_hash.update(byte_block)
#     return sha256_hash.hexdigest()

# calculate_sha256('benchmark/adult_private_for_demo.csv')

In [9]:
loader = PETsARD.Loader(
    filepath='benchmark://adult',
)
print(loader.data.head(1))

Loader - Benchmarker : Success download the benchmark dataset in https://petsard-benchmark.s3.amazonaws.com/adult.csv.
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [10]:
loader = PETsARD.Loader(
    filepath='benchmark://adult-private',
)
print(loader.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
